# Automatic form filling

Required packages:
- pip install pypdf2
- pip install pymupdf
- pip install openai
- pip install pandas
- pip install openpyxl
- pip install numpy
- pip install opencv-python
- pip install Pillow
- pip install python-docx

*Disclaimer - Considering the API privacy, Please replace api_key and selected_model from the config.py to your custom LLM api key and selected_model

In [ ]:
from config import api_key, selected_model

api key of chat gpt.

In [ ]:
# Importing the required Packages
import requests
import openai
import fitz
import pandas as pd
import os
import openpyxl
import numpy as np
import io
import cv2
from PIL import Image
from math import sqrt
from docx import Document

In [ ]:
pip install python-docx


In [ ]:
pip install PyMuPDF


In [ ]:
pip install openai


In [ ]:
#extra_questions is the input field that is used to update the unrecognised questions after an initial run
extra_questions = []

using the llm model to detect the files which it wont properly detect, so those questions will be passed in the etra ques, not only the detected questions.

## The Input resources (should be edited as per the purpose)

- Data

In [ ]:
file_path = '/content/excell form data.xlsx'#The file which consist of the data that should have to be entered in the form

- PDF filepaths

In [ ]:
pdf_path = '/content/BCU-application-form.pdf'#The particular path of the PDF file you want to fill
output_pdf_path = '/content/output project .pdf'#the particlar path of a empty PDF file where you want save the filled PDF

- Word filepath

In [ ]:
word_path = '/content/authorised_interruption_of_study_application_1.docx'#The particular path of the Word file you want to fill
output_path = 'fully filled'#you can give any desired name that you want for the filled form

- Excel Filepath

In [ ]:
excel_path = '/content/StudentAnnualPermitApplicationForm (1).xlsx'#The particular path of the Excel file you want to fill

### 1. Data Loading

In [ ]:
def load_data(file_path):
    # Determining the file extension
    _, file_extension = os.path.splitext(file_path)

    # checking the file extension to load it as per the extention
    if file_extension in ['.xls', '.xlsx']:
        # It's an Excel file
        df = pd.read_excel(file_path)
    elif file_extension in ['.csv', '.txt']:
        # It's a CSV or a text file
        df = pd.read_csv(file_path)
    else:
        raise ValueError("Unsupported file type")

    # Convert the DataFrame to a dictionary (assuming you want to use all data as context)
    data_context_dict = df.to_dict(orient='records')[0]  # Taking the first row as an example

    return data_context_dict

#Loading the particular data you had given in the input resources
data_context_dict = load_data(file_path)
# Convert the dictionary to a string context to make it similar to a statement about the particular person with the given data baout him
data_context = '. '.join([f"{key} is {value}" for key, value in data_context_dict.items()])

### 2. General Functions for Form Filling

In [ ]:
#establishing the API connection using the API key and setting the temperature as 0.7
def call_chat_gpt(prompt, api_key, selected_model):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": selected_model,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }
    response = requests.post(url, json=payload, headers=headers)
    return response.json()
#To extract the content from the generated answers , the function answer_cleanup had been made
def answer_cleanup(output):
    try:
        return output['choices'][0]['message']['content']
    except (KeyError, IndexError, TypeError):
        return "Error in processing the response."

#Now this function is trying to identify the questions from the extracted text of document that is already loaded
def identify_keywords_with_llm(pdf_text, api_key, selected_model):
#The specific prompt to generate questions
    prompt = f"""
Given the text, identify and list only the explicit questions and statements that indicate a requirement for user input. This includes:

- Direct questions that clearly ask for specific information (e.g., "What is your name?").
- Statements or phrases that directly imply a requirement for a response or filling a blank (e.g., "Name of the applicant: _______").
- Explicit prompts for specific types of information, irrespective of whether they are formatted as questions or statements (e.g., "Date of birth:", "Briefly describe your experience with...").

Exclude:
- General descriptive text or narrative explanations.
- Any content that does not directly request user input or response.
Important formatting requirements that should be implemented for the output:
- Remove any leading characters such as dashes ('-'),colon(:), list numbers, or all punctuation from the beginning of each line to focus solely on the content.
- Retain the original text format as much as possible beyond the initial character removal to ensure clarity and ease of reading.
- Extract the clean text of each question or statement requesting user input, excluding any extraneous elements or formatting characters.
- Include all variations of input requests, whether they are formatted as direct questions, implied statements, or instructional texts, without the initial formatting characters.

Please process the text with these instructions:

{pdf_text}
"""
#Calling the gpt with the given prompt
    llm_response = call_chat_gpt(prompt, api_key, selected_model)
    #doing the answer clean up to extarct the content only
    response = answer_cleanup(llm_response)
    #spliting the answer from where it has '\n'(assumming that as an end of a line)
    keywords_from_llm_1 = response.split('\n')
    #sometimes the questions generaed might come with specific '-' so here I am striping that from each
    keywords_from_llm = [keyword.lstrip('- ').strip() for keyword in keywords_from_llm_1]
    return keywords_from_llm

## PDF Files

### 3. Functions for PDF Filling Processes

In [ ]:
#Function which is capable of extracting the text from a pdf using fitz library
def extract_all_text(pdf_path):
    # Open the PDF file
    doc = fitz.open(pdf_path)

    all_text = ""  # Initialize a variable to store all the extracted text

    # Iterate through each page in the PDF
    for page in doc:
        # Extract text from the current page
        page_text = page.get_text()
        all_text += page_text + "\n"  # Append the text from this page, add a newline as separator

    return all_text
#Now generating the answers for the specific questions extracted from the pdf extarcted text
def generate_answer_with_context(context, question, api_key, selected_model):
    # This is the prompt for answer generation explicitly for PDF extracted text
    prompt = prompt = f"Based on the following details: {context}, if the information is explicitly provided, what is the concise answer (preferably a single word or a short phrase) for '{question}'? Keep the response brief. If not enough information is available for a question, just respond with ''."
    #calling my gpt to generate the answers with the prompt I had given
    response = call_chat_gpt(prompt, api_key, selected_model)
    cleaned_response = answer_cleanup(response).strip()
#In PDF forms only I am replacing the insufficient data with an empty string("")
    if cleaned_response.lower() == "":
        return None
    else:
        return cleaned_response

#Now I am creating function specifically for the box detection in the pdf forms
#This function is converting my pdf pages into images by loading each of the pages
def extract_images_from_pdf(pdf_path):
    images = []
    doc = fitz.open(pdf_path)
    for page in doc:
        pix = page.get_pixmap()
        img_bytes = pix.tobytes(output="png")
        image = Image.open(io.BytesIO(img_bytes)).convert("RGB")
        image_np = np.array(image)
        # Convert RGB to BGR for OpenCV
        image_np = image_np[:, :, ::-1].copy()
        images.append(image_np)
    return images

#Now I am doing the greyscale conversion, guassian blur, edge detection and finding the contours to find the coordinates of the boxes in the form
def find_boxes_in_image(image):
    boxes = []
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (3, 3), 0)
        edges = cv2.Canny(blur, 100, 200)
        contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            approx = cv2.approxPolyDP(contour, 0.01 * cv2.arcLength(contour, True), True)
            if len(approx) == 4:
                # approx contains the coordinates of the 4 corners
                corners = approx.reshape(-1, 2)
                boxes.append(corners.tolist())
    except cv2.error as e:
        print(f"Error in finding boxes: {e}")
    return boxes

#my previous function was detecting some unnneccesary small boxes along with actual boxes
#So here I am calculating the area of all the boxes
def calculate_rectangle_area(box):
    xs = [point[0] for point in box]
    ys = [point[1] for point in box]
    width = max(xs) - min(xs)
    height = max(ys) - min(ys)
    return width * height

#The next step is to find the coordinates of the identifird questions by LLM
def extract_text_coordinates(pdf_path, keywords):

    doc = fitz.open(pdf_path)  # Open the PDF
    text_info = []  # List to hold text and coordinates

    for page_num in range(len(doc)):  # Iterate through each page
        page = doc.load_page(page_num)  # Load the current page
        text_instances = page.get_text("dict")["blocks"]  # Extract text instances as dictionaries

        for instance in text_instances:
            if 'lines' in instance:  # Check if the block contains lines of text
                for line in instance['lines']:
                    for span in line['spans']:  # Iterate through each text span in the line
                        text = span['text']
                        if any(keyword.lower() in text.lower() for keyword in keywords):
                            bbox = span['bbox']  # Bounding box of the text

                            # Calculate all four corners based on bbox
                            corners = [
                                (bbox[0], bbox[1]),  # Bottom-left
                                (bbox[2], bbox[1]),  # Bottom-right
                                (bbox[2], bbox[3]),  # Top-right
                                (bbox[0], bbox[3])   # Top-left
                            ]
                            text_info.append({
                                "page": page_num,
                                "text": text,
                                "corners": corners  # List of all four corner coordinates
                            })

    return text_info


#Now using the universal mathematical concept euclidean distance to find the distance between each question and all the answer boxes to find the shortest one

def calculate_euclidean_distance(point1, point2):

    return sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)
#Using the below function I am assigning the closest box which means the one box where the answers should go for each question by calculating the eucledean distance as mentioned above. And assigning the short distanced one as the corresponding box

def find_closest_box(text_coordinates, box_coordinates):
    closest_boxes = []

    for text_item in text_coordinates:
        question_page = text_item['page']  # Get the page number of the text item
        # Assuming the corners are ordered as bottom-left, bottom-right, top-right, top-left
        text_top_right = text_item['corners'][2]  # Top right corner of the text

        min_distance = float('inf')
        closest_box = None  # Initialize closest box to None

        # Ensure we're looking at the correct page's boxes
        if question_page < len(box_coordinates):
            for box in box_coordinates[question_page]:
                # Correctly access the top left corner of the box directly
                # Assuming corners order: bottom-left, bottom-right, top-right, top-left
                box_top_left = box[3]  # Direct access to the top left corner

                # Calculate Euclidean distance from text top right to box top left because we want to calculate the most closest portion between a question and its corresponding box
                distance = calculate_euclidean_distance(text_top_right, box_top_left)

                # Update the closest box if the current box is closer
                if distance < min_distance:
                    min_distance = distance
                    closest_box = box

        # Append the result for the current question to the list
        closest_boxes.append({
            'question': text_item['text'],
            'closest_box': closest_box,
            'page': question_page
        })

    return closest_boxes

#Now I want to print the generated answers into the pdf and want to save it into the output file path
def save_generated_pdf(input_pdf_path, question_answer_boxes, answers, output_pdf_path, closest_boxes):
    doc = fitz.open(input_pdf_path)

    for item in closest_boxes:
        question = item['question'].strip(':')  # Remove colon for matching
        closest_box = item['closest_box']
        page_number = item['page']
        answer = answers.get(question, '')  # Get the answer for the question

        if answer:
            # Assuming closest_box is a list of points [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]
            # and we'll use the top-left corner to position the text
            # Sorting by x to get left points, then by y to get the topmost point
            top_left_corner = sorted(closest_box, key=lambda point: (point[0], point[1]))[0]
            # Adjusting the position slightly to the right (x + 2) and down (y + 10) for better visibility
            position = (top_left_corner[0] + 2, top_left_corner[1] + 10)

            page = doc.load_page(page_number)
            # Insert the text at the adjusted position
            page.insert_text(position, answer, fontsize=10)
#lastly saving the file to the output file path
    doc.save(output_pdf_path)

### 4. Implementing the functions to fill the PDF form

In [ ]:
# Extract all text from the PDF
extracted_text = extract_all_text(pdf_path)


# Call the LLM with the extracted PDF text to identify the questions
keywords_from_llm = identify_keywords_with_llm(extracted_text, api_key, selected_model)

# Including the extra question if there is anything in that list
if extra_questions:
    keywords_from_llm.extend(extra_questions)

# Generating answers for each keyword
answers = {keyword: generate_answer_with_context(data_context_dict, keyword, api_key, selected_model) for keyword in keywords_from_llm}

In [ ]:
 keywords_from_llm

['Course Title:',
 'Proposed start date:',
 'Proposed Year/Level of Entry:',
 'Date of birth:',
 'First name(s):',
 'Maiden or any other name(s) that you have been known by:',
 'Surname/family name:',
 'Permanent address:',
 'Correspondence address (if different):',
 'Daytime telephone:',
 'Evening telephone (if different):',
 'Mobile:',
 'Email address:',
 'Nationality:',
 'If not born in the UK please state date of arrival to UK:',
 'Area of permanent residence:',
 'If you are a member of a Professional Body, please give its name and your Registration Number:',
 'Have you ever studied in the UK before? (If yes, please include a copy of all visas)',
 'What level was your previous study in the UK (please tick all that apply)?',
 'Have you ever studied at Birmingham City University before?',
 'Please enter details of the highest level of qualification you currently hold.',
 'If you have a 10 digit Unique Learner Number (ULN), please enter it in the box provided.',
 'If you are an overse

#### 4.1 Using Image Analysis in box detection and filling the form

In [ ]:
# Extracting the images from PDF
images = extract_images_from_pdf(pdf_path)

# Initialize an empty list to hold the box coordinates for all images
box_coordinates = []

# Find answer box coordinates for each image and add them to the box_coordinates list
for i, image in enumerate(images):
    boxes = find_boxes_in_image(image)
    box_coordinates.append(boxes)   # Add the boxes for this image to the main list

filtered_box_coordinates = []#the boxes to append the filtered boxes as per the area

for page_boxes in box_coordinates:
    # Filter boxes based on their calculated area by assigning 50 as a threshold area
    filtered_boxes = [box for box in page_boxes if calculate_rectangle_area(box) >= 50]
    filtered_box_coordinates.append(filtered_boxes)

#Extracting the coordinates of the questions identified by the LLMs
text_coordinates = extract_text_coordinates(pdf_path,keywords_from_llm)
#Finding the closest boxes by calculating the euclidean distances and assigning the short distanced box as the corresponding box
closest_boxes = find_closest_box(text_coordinates, filtered_box_coordinates)
#inserting the answers in the corresponding boxes and saving the file
save_generated_pdf(pdf_path, closest_boxes, answers, output_pdf_path, closest_boxes)

##  Word Files

### 5. Functions for Word Filling Processes

In [ ]:
#so in here the word files might be in either .docx format or .doc format this function convert the doc to docx to make it work on any formats and making it as much as flexible
def convert_doc_to_docx(doc_path):
    # Ensure the path is absolute
    doc_path = os.path.abspath(doc_path)
    # Create a .docx path for the converted file
    docx_path = os.path.splitext(doc_path)[0] + ".docx"

    # Start Word application
    word = win32.gencache.EnsureDispatch('Word.Application')
    doc = word.Documents.Open(doc_path)
    # Save the document in .docx format
    doc.SaveAs2(docx_path, FileFormat=16)  # 16 represents wdFormatXMLDocument
    doc.Close()
    word.Quit()

    return docx_path
#So now I am extracting the texts of the word file especially from parigrph and tables
def extract_text_in_order(file_path):
    # Check if file is .doc, convert to .docx
    if file_path.endswith('.doc'):
        print("Converting .doc to .docx")
        file_path = convert_doc_to_docx(file_path)

    doc = Document(file_path)
    extracted_text = []

    # Extract text from paragraphs
    for para in doc.paragraphs:
        extracted_text.append(para.text)

    # Extract text from tables
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                extracted_text.append(cell.text)

    return extracted_text
#So this function is precisely for generating answers for the questions generated from the word extracted text
def generate_answer_with_context(context, question, api_key, selected_model):
    # Update the prompt to specify that the response should be "Insufficient data" if not enough context is provided
    prompt = (f"Based on the following details: {context}, for the question '{question}', "
                  "what is the concise answer (preferably a single word or a short phrase)? "
                  "If not enough information is available, respond with 'Insufficient data'. "
                  "For imaginative or creative questions, where crafting a story or providing an imaginative answer is suitable,and do not condider the condition ogf making the answer concise while you create a story for a question "
                  "use the available context to craft a compelling, creative response that aligns with the theme or nature of the question.")
# Use this 'prompt' with your LLM to generate the response.

    response = call_chat_gpt(prompt, api_key, selected_model)
    cleaned_response = answer_cleanup(response).strip()

    # Check if the response is "Insufficient data" or similar to indicate not to generate an answer
    if cleaned_response.lower() == "":
        return None  # Or any suitable indicator for no answer due to insufficient context
    else:
        return cleaned_response

#So this function is helping as to manupulate the word files ,initially by converting the doc into docx because the docx library can only do manupulation if the document is in . docx format
def fill_in_answers(doc_path, answers_dict, output_file_name):
    # Check if the file is a .doc file, and convert it to .docx if so
    if doc_path.endswith(".doc"):
        print("Converting .doc to .docx")
        doc_path = convert_doc_to_docx(doc_path)

    doc = Document(doc_path)

    # Process paragraphs in the document body
    for i, para in enumerate(doc.paragraphs):
        for keyword, answer in answers_dict.items():
            if keyword.lower() in para.text.lower():  # Case insensitive search for the keyword
                # Check if the next paragraph is empty and we are not at the last paragraph
                if i + 1 < len(doc.paragraphs) and not doc.paragraphs[i + 1].text.strip():
                    # Insert the answer into the next paragraph
                    doc.paragraphs[i + 1].text = answer

    # Process each table in the document
    for table in doc.tables:
        for row in table.rows:
            for i, cell in enumerate(row.cells):
                for keyword, answer in answers_dict.items():
                    if keyword.lower() in cell.text.lower():
                        # Attempt to find the next empty cell in the row
                        if i + 1 < len(row.cells) and not row.cells[i + 1].text.strip():
                            row.cells[i + 1].text = answer

    # Construct the output path with .docx extension
    output_dir = os.path.dirname(doc_path)  # Use the same directory as the input file
    output_path = os.path.join(output_dir, output_file_name + ".docx")

    # Save the document to the specified output path
    doc.save(output_path)

### 6. Implementing the functions to fill the word form

In [ ]:

# Extracting all texts from the word file
extracted_text = extract_text_in_order(word_path)


# Calling the gpt to identify the question from the extracted text
keywords_from_llm = identify_keywords_with_llm(extracted_text, api_key, selected_model)

# Including the extra question if there is anything in that list
if extra_questions:
    keywords_from_llm.extend(extra_questions)

# Generate answers for each keyword
answers = {keyword: generate_answer_with_context(data_context_dict, keyword, api_key, selected_model) for keyword in keywords_from_llm}
#Filling the answers into the next empty space of each identified question
fill_in_answers(word_path,answers,output_path)

## Excel Spreadsheet

### 7. Functions for Excel filling processes

In [ ]:
#The function specifically for generating answers for the questions extracted from excel file
def generate_answer_with_context(context, question, api_key, selected_model):
    # Update the prompt to specify that the response should be "Insufficient data" if not enough context is provided
    prompt = f"Based on the following details: {context}, answer the question '{question}' as follows: If the question is a direct query seeking specific information (e.g., 'What is the capital of France?'), provide a concise answer (preferably a single word or a short phrase). If the question is analytical and seeks a deeper explanation (e.g., 'How does photosynthesis work?'), provide a detailed and analytical answer. For yes/no questions (e.g., 'Is the sky blue?'), respond with 'Yes' or 'No' and include a brief justification. If not enough information is available to accurately answer the question, respond with 'Insufficient information.'and only provide answers for a question once and in the output I just need the answers to be alone not with the questions"


#calling the gpt to generate answers foe each question
    response = call_chat_gpt(prompt, api_key, selected_model)
    cleaned_response = answer_cleanup(response).strip()

    # Check if the response is "Insufficient data" or similar to indicate not to generate an answer
    if cleaned_response.lower() == "":
        return None  # Or any suitable indicator for no answer due to insufficient context
    else:
        return cleaned_response
    #some times the files might be in csv format
    # Function to convert CSV to XLSX if necessary and return the path to the XLSX file
def ensure_xlsx(file_path):
    dir_name, file_name = os.path.split(file_path)
    base_name, extension = os.path.splitext(file_name)

    # If the file is a CSV, convert it to XLSX
    if extension.lower() == '.csv':
        df = pd.read_csv(file_path)
        new_file_path = os.path.join(dir_name, base_name + '.xlsx')
        df.to_excel(new_file_path, index=False)  # Save the DataFrame as an XLSX file
        return new_file_path
    elif extension.lower() in ['.xls', '.xlsx']:
        return file_path
    else:
        raise ValueError("Unsupported file type")

### 8. Implementing the functions to fill the Excel form

In [ ]:
#loading the data using pandas
data = pd.read_excel(excel_path)
#converting the loaded data into string values
data_string = '\n'.join(data.applymap(str).stack().tolist())
#calling gpt to identify the questions from the data string
keywords_from_llm = identify_keywords_with_llm(data_string, api_key, selected_model)

# Including the extra question if there is anything in that list
if extra_questions:
    keywords_from_llm.extend(extra_questions)

# Generate answers for each keyword
answers = {keyword: generate_answer_with_context(data_context_dict, keyword, api_key, selected_model) for keyword in keywords_from_llm}
# Ensure the file is in XLSX format
xlsx_file_path = ensure_xlsx(excel_path)
#From here onwards using the openpyxl library I am trying to fill the answers to the corresponding empty cells to each questions
wb = openpyxl.load_workbook(xlsx_file_path)
sheet = wb.active

# Iterate over the rows and columns to find the questions
for row in range(1, sheet.max_row + 1):
    for col in range(1, sheet.max_column + 1):
        cell = sheet.cell(row=row, column=col)
        cell_value = cell.value
        if cell_value in answers:
            # Once the question is found, search for the next empty cell in the same row
            for answer_col in range(col + 1, sheet.max_column + 2):
                next_cell = sheet.cell(row=row, column=answer_col)
                if next_cell.value is None or isinstance(next_cell, openpyxl.cell.cell.MergedCell):
                    # Find the top-left cell of the merged range that includes the next_cell
                    for range_ in sheet.merged_cells.ranges:
                        if next_cell.coordinate in range_:
                            top_left_cell = sheet.cell(range_.min_row, range_.min_col)
                            top_left_cell.value = answers[cell_value]
                            break
                    else:
                        # If next_cell is not a merged cell, just assign the value directly
                        next_cell.value = answers[cell_value]
                    break  # Exit the loop after inserting the answer

# Define a new file name by appending "filled" before the file extension in the original file path
dir_name, file_name = os.path.split(excel_path)
base_name, extension = os.path.splitext(file_name)
new_file_name = base_name + "_filled" + extension
new_file_path = os.path.join(dir_name, new_file_name)

# Save the updated workbook to the new file path
wb.save(new_file_path)


## Appendix
*Disclaimer - In the supplementary materials, I've included two separate folders: one contains all the forms I've filled, and the other holds the original formats of these forms. For demonstration purposes, I've provided a single completed form in each of the three formats. This arrangement will give you access to both the finalized forms and their initial templates.